# isoration Forest 時系列異常検知

### 必要なモジュールのimport

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

### セグメントの切り出し関数

In [ ]:
def segdata(lst, dim):
    emb = np.empty((0,dim), float)
    for i in range(lst.size - dim + 1):
        tmp = np.array(lst[i:i+dim])[::-1].reshape((1,-1)) #セグメントの切り出し，時系列反転，appendのための2次ベクトル化
        emb = np.append( emb, tmp, axis=0)
    return emb

### データ読み込み，パラメータ設定
 Keoghらの心電図のデータ  http://www.cs.ucr.edu/~eamonn/discords/qtdbsel102.txt
 Keogh, E., Lin, J. and Fu, A.: HOT SAX : Efficiently Finding the Most Unusual Time Series Subsequence, in Proceedings of the Fifth IEEE International Conference on Data Mining, ICDM 05, pp.226-233.

In [ ]:
!wget "www.dropbox.com/s/x3fmb9mxr4xkip3/qtdbsel102.txt"

In [ ]:
LEN=3000  #学習テストそれぞれの分析区間
WLEN=150 #セグメントのサイズ
SP=0         #学習用データの開始点
AP=LEN   #テスト用データの開始点　このデータの場合 1250ポイント付近(データ全体では4250ポイント)に異常がある

In [ ]:
data = np.loadtxt("qtdbsel102.txt",delimiter="\t")
print("総データ数:",data.shape[0],"  次元数:",data.shape[1])

#元データは3次元の時系列，3次のデータ．
#ここでは１次元データを扱うため，index=2 のデータを取得し，これを学習/テストデータに分割
train_org = data[SP:SP+LEN, 2]  #学習用データ
test_org  = data[AP:AP+LEN, 2]  #テスト用データ

plt.plot(train_org)
plt.show()
plt.plot(test_org)
plt.show()


### 窓関数の設定と切り出し

In [ ]:
seglen = WLEN   #Window size
#winlenの単位で1ポイントずつずらした2次元表現に変換
train = segdata(train_org, seglen)
test =  segdata(test_org, seglen)
print(train.shape)
print(test.shape)

## isolation Forest（学習データ）

In [ ]:
outliers_fraction = 0.0001
rng = np.random.RandomState(123)
isof = IsolationForest(contamination=outliers_fraction,
                          max_samples="auto",
                          random_state=rng,
                          n_estimators=100)

isof.fit(train) #学習
score_pred = isof.decision_function(train) #判定スコア
pred = isof.predict(train) #判定
print(pred)
outlier_rows = [i for i in range(len(pred)) if pred[i]==-1]
print(outlier_rows)

In [ ]:
for c in outlier_rows:
    plt.axvspan(c, c+WLEN, color = "skyblue")
plt.plot(train_org)
plt.show()

## 未知データに対する評価

In [ ]:
score_pred = isof.decision_function(test)
pred = isof.predict(test)
outlier_rows = [i for i in range(len(pred)) if pred[i]==-1]
print(outlier_rows)
    

In [ ]:
plt.plot(score_pred)

In [ ]:
for c in outlier_rows:
    plt.axvspan(c, c+WLEN, color = "skyblue")
plt.plot(test_org)
plt.plot(score_pred)
plt.show()